# DAIN Colab alpha

Credits to original Colab file: 
https://github.com/baowenbo/DAIN/issues/44

My fork:
https://github.com/styler00dollar/DAIN

Enhancement by [Styler00Dollar](https://github.com/styler00dollar) aka "sudo rm -rf / --no-preserve-root#8353" on discord and [Alpha](https://github.com/AlphaGit). Please do not run this command in your linux terminal. It's rather meant as a joke.

A simple guide:
- Copy the .ipynb-file to your drive.
- Create a folder inside of Google Drive named "DAIN"
- Change the configurations in the next cell
- Run cells one by one

Stuff that should be improved:
- Gif (RGBA24 or alpha in general) is currently not supported.
- Adding configuration to select speed.
- Detect scenes to avoid interpolating scene-changes
- Auto-resume
- Copy `start_frame` - `end_frame` audio from original input to final output

In [0]:
################# Configurations ############################

# Use the values in here to configure what you'd like DAIN to do.

# Input file: Path (relative to the root of your Google Drive) to the input file.
# For instance, if you save your "example.mkv" file in your Google Drive, inside a "videos" folder, the path would be:
# videos/example.mkv
INPUT_FILEPATH = "DAIN/batmanIn.mp4"

# Output file path: path (relative to the root of your Google Drive) for the output file.
# Extension should always be MP4.
OUTPUT_FILE_PATH = "DAIN/batmanOut.mp4"

################# Optional configurations ############################

# Target FPS = how many frames per second should the result have. This will determine how many intermediate images are
# interpolated.
TARGET_FPS = 60

# Frame input directly
# Use a path that is in your GDrive if you already have the list of frames in the format 00001.png, 00002.png, etc.
# Your GDrive is located at `/content/gdrive/My Drive/`
FRAME_INPUT_DIR = '/content/DAIN/input_frames'

# Frame output directory
# Use a location in your GDrive if you want the generated frames stored to your Google Drive.
# Your GDrive is located at `/content/gdrive/My Drive/DAIN/tmp`
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames'

In [3]:
# Connect Google Drive

from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Google Drive connected.


In [4]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# P100: 16GB (Can handle 720p. 1080p does not work and you will get an out-of-memory error.)
# P4: 8GB (Not tested.)
# K80: 8GB (Not tested.)

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P4, 418.67, 7611 MiB


# Install dependencies.

This next step may take somewhere between 15-20 minutes.

Look for the "Restart" warning when it finishes, but **do not restart** the notebook.


In [5]:
from IPython.display import clear_output
!rm -rf /content/DAIN
!git clone https://github.com/styler00dollar/Colab-DAIN /content/DAIN

# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
#clear_output()
print("Building #1 done.")

# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
#clear_output()
print("Building #2 done.")

# Downloading pre-trained models
%cd /content/DAIN
!mkdir model_weights MiddleBurySet
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

%cd /content/DAIN/MiddleBurySet
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
!unzip other-color-allframes.zip

!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
!unzip other-gt-interp.zip
#clear_output()

!CUDA_VISIBLE_DEVICES=0

# Fix scipy
# DO NOT RESTART YOUR COLAB!! Yes, there is red text and a warning, but it works.

!pip install --force-reinstall scipy==1.0.0

Cloning into '/content/DAIN'...
remote: Enumerating objects: 636, done.
remote: Total 636 (delta 0), reused 0 (delta 0), pack-reused 636
Receiving objects: 100% (636/636), 230.82 KiB | 321.00 KiB/s, done.
Resolving deltas: 100% (358/358), done.
/content/DAIN/my_package
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating mindepthflowprojection_cuda.egg-info
writing mindepthflowprojection_cuda.egg-info/PKG-INFO
writing dependency_links to mindepthflowprojection_cuda.egg-info/dependency_links.txt
writing top-level names to mindepthflowprojection_cuda.egg-info/top_level.txt
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'mindepthflowprojection_cuda' extension
creating build
creating build/temp.linux-x86_64-3.

In [20]:
# Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

# TODO: Work with this to remove the alpha channel
#%shell ffmpeg -i '/content/DAIN/{filename}' -filter_complex 'split [rgb_in][alpha_in]; [rgb_in][out];' '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '/content/DAIN/{filename}' '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())
print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

Input FPS: 11.11111111111111
11 frame PNGs generated.


In [21]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python colab_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

/content/DAIN
revise the unique id to a random numer 32841
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/32841-Sun-Mar-22-01:38/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, end_frame=11, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, frame_input_dir='/content/DAIN/input_frames', frame_output_dir='/content/DAIN/output_frames', log='./model_weights/32841-Sun-Mar-22-01:38/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/32841-Sun-Mar-22-01:38', save_which=1, seed=1, start_frame=1, time_step=0.18518518518518517, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 4 frames
The testing model weight is: ./model_weights/best.pth
/usr/local/lib/pyth

In [24]:
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena